In [1]:
!pip install -q transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
!huggingface-cli login --token hf_

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer

In [4]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics2(preds, labels):
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average="weighted")
    recall = recall_score(labels, preds, average="weighted")
    f1 = f1_score(labels, preds, average="weighted")
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

# "minoosh/bert-clf-crossencoder-kl_divergence"

In [5]:
model_name = "minoosh/bert-clf-crossencoder-kl_divergence"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Initialize the Trainer
trainer = Trainer(model=model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/882 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [6]:
# Load dataset
dataset = load_dataset("minoosh/EPITOME_pairs")
#tokenizer = trainer.tokenizer

# Preprocess data for the cross-encoder model by concatenating text1 and text2 with [SEP]
def preprocess_function(examples):
    # Concatenate both texts with a [SEP] token in between
    encodings = tokenizer(examples['text1'], examples['text2'], truncation=True, padding=True, max_length=512)
    encodings['labels'] = examples['label']  # Add labels
    return encodings

# Apply tokenization
#tokenized_train = dataset['train'].map(preprocess_function, batched=True)
tokenized_test = dataset['test'].map(preprocess_function, batched=True)
#tokenized_val = dataset['validation'].map(preprocess_function, batched=True)



# Set format for PyTorch

#tokenized_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
#tokenized_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

README.md:   0%|          | 0.00/588 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/660k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/100k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/88.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2467 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/308 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/309 [00:00<?, ? examples/s]

Map:   0%|          | 0/308 [00:00<?, ? examples/s]

In [7]:
prediction = trainer.predict(tokenized_test)
prediction

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


PredictionOutput(predictions=array([[-1.1600488 , -0.7891972 ,  1.2061149 ,  0.7121975 ],
       [ 0.92508686,  1.4973122 , -1.2848904 , -0.89222693],
       [ 1.201261  ,  1.0581589 , -0.47902712, -1.2772454 ],
       ...,
       [ 1.6557971 , -0.7263769 ,  1.1232517 , -2.0013373 ],
       [-0.739866  , -1.6214441 ,  2.321577  , -0.6545602 ],
       [ 2.0248005 , -0.00261627,  0.31250072, -2.0748637 ]],
      dtype=float32), label_ids=array([0, 1, 0, 0, 1, 2, 0, 3, 2, 2, 0, 0, 2, 2, 2, 1, 0, 1, 1, 2, 0, 3,
       2, 1, 3, 0, 0, 2, 3, 2, 2, 2, 0, 0, 2, 2, 2, 3, 2, 3, 1, 2, 2, 3,
       0, 1, 2, 2, 0, 2, 2, 3, 3, 2, 1, 3, 0, 1, 3, 2, 2, 1, 2, 2, 2, 0,
       0, 2, 2, 1, 0, 0, 2, 2, 2, 1, 2, 3, 2, 1, 0, 1, 0, 2, 0, 2, 2, 1,
       2, 2, 2, 0, 2, 1, 0, 2, 0, 1, 2, 0, 3, 2, 2, 1, 3, 2, 1, 1, 2, 3,
       3, 3, 3, 2, 1, 2, 0, 3, 0, 3, 3, 3, 3, 2, 2, 0, 0, 2, 1, 2, 0, 0,
       2, 0, 0, 2, 2, 2, 3, 0, 2, 2, 2, 2, 3, 2, 2, 0, 2, 2, 2, 2, 2, 2,
       0, 2, 2, 2, 2, 1, 3, 0, 1, 0, 1, 0, 0, 1, 

In [9]:
actual_labels = tokenized_test['labels'].tolist()
predicted_classes = list(prediction.predictions.argmax(axis=1))
res = compute_metrics2(actual_labels, predicted_classes)

res

{'accuracy': 0.5844155844155844,
 'precision': 0.6027174337276708,
 'recall': 0.5844155844155844,
 'f1': 0.5868961556072686}

# "minoosh/bert-clf-crossencoder-focal_loss"

In [5]:
model_name = "minoosh/bert-clf-crossencoder-focal_loss"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Initialize the Trainer
trainer = Trainer(model=model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/882 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [6]:
# Load dataset
dataset = load_dataset("minoosh/EPITOME_pairs")
#tokenizer = trainer.tokenizer

# Preprocess data for the cross-encoder model by concatenating text1 and text2 with [SEP]
def preprocess_function(examples):
    # Concatenate both texts with a [SEP] token in between
    encodings = tokenizer(examples['text1'], examples['text2'], truncation=True, padding=True, max_length=512)
    encodings['labels'] = examples['label']  # Add labels
    return encodings

# Apply tokenization
#tokenized_train = dataset['train'].map(preprocess_function, batched=True)
tokenized_test = dataset['test'].map(preprocess_function, batched=True)
#tokenized_val = dataset['validation'].map(preprocess_function, batched=True)



# Set format for PyTorch

#tokenized_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
#tokenized_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

README.md:   0%|          | 0.00/588 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/660k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/100k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/88.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2467 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/308 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/309 [00:00<?, ? examples/s]

Map:   0%|          | 0/308 [00:00<?, ? examples/s]

In [7]:
prediction = trainer.predict(tokenized_test)
prediction

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


PredictionOutput(predictions=array([[ 0.3585632 , -1.254249  ,  1.4455051 ,  0.01605132],
       [ 0.44165954,  1.0037017 , -0.14564386, -0.8646102 ],
       [-0.17403544,  0.30213526, -0.37435156,  0.6291068 ],
       ...,
       [ 1.2942659 , -0.38737556,  1.3200699 , -1.734803  ],
       [ 0.69948995, -0.59273905,  1.5678897 , -1.5955803 ],
       [ 1.4429803 , -0.16045286,  1.088097  , -1.8013549 ]],
      dtype=float32), label_ids=array([0, 1, 0, 0, 1, 2, 0, 3, 2, 2, 0, 0, 2, 2, 2, 1, 0, 1, 1, 2, 0, 3,
       2, 1, 3, 0, 0, 2, 3, 2, 2, 2, 0, 0, 2, 2, 2, 3, 2, 3, 1, 2, 2, 3,
       0, 1, 2, 2, 0, 2, 2, 3, 3, 2, 1, 3, 0, 1, 3, 2, 2, 1, 2, 2, 2, 0,
       0, 2, 2, 1, 0, 0, 2, 2, 2, 1, 2, 3, 2, 1, 0, 1, 0, 2, 0, 2, 2, 1,
       2, 2, 2, 0, 2, 1, 0, 2, 0, 1, 2, 0, 3, 2, 2, 1, 3, 2, 1, 1, 2, 3,
       3, 3, 3, 2, 1, 2, 0, 3, 0, 3, 3, 3, 3, 2, 2, 0, 0, 2, 1, 2, 0, 0,
       2, 0, 0, 2, 2, 2, 3, 0, 2, 2, 2, 2, 3, 2, 2, 0, 2, 2, 2, 2, 2, 2,
       0, 2, 2, 2, 2, 1, 3, 0, 1, 0, 1, 0, 0, 1, 

In [8]:
actual_labels = tokenized_test['labels'].tolist()
predicted_classes = list(prediction.predictions.argmax(axis=1))
res = compute_metrics2(actual_labels, predicted_classes)

res

{'accuracy': 0.5746753246753247,
 'precision': 0.6310700757575757,
 'recall': 0.5746753246753247,
 'f1': 0.5891196106162209}

# "minoosh/bert-clf-crossencoder-cross_entropy"

In [5]:
model_name = "minoosh/bert-clf-crossencoder-cross_entropy"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Initialize the Trainer
trainer = Trainer(model=model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/882 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [6]:
# Load dataset
dataset = load_dataset("minoosh/EPITOME_pairs")
#tokenizer = trainer.tokenizer

# Preprocess data for the cross-encoder model by concatenating text1 and text2 with [SEP]
def preprocess_function(examples):
    # Concatenate both texts with a [SEP] token in between
    encodings = tokenizer(examples['text1'], examples['text2'], truncation=True, padding=True, max_length=512)
    encodings['labels'] = examples['label']  # Add labels
    return encodings

# Apply tokenization
#tokenized_train = dataset['train'].map(preprocess_function, batched=True)
tokenized_test = dataset['test'].map(preprocess_function, batched=True)
#tokenized_val = dataset['validation'].map(preprocess_function, batched=True)



# Set format for PyTorch

#tokenized_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
#tokenized_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

README.md:   0%|          | 0.00/588 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/660k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/100k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/88.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2467 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/308 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/309 [00:00<?, ? examples/s]

Map:   0%|          | 0/308 [00:00<?, ? examples/s]

In [7]:
prediction = trainer.predict(tokenized_test)
prediction

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


PredictionOutput(predictions=array([[-0.768501  , -1.0799723 ,  2.5389264 ,  0.44930747],
       [ 1.4267197 ,  0.9082134 , -0.07638107, -1.4540707 ],
       [ 1.8063105 ,  0.29711446, -0.12753932, -1.5108252 ],
       ...,
       [ 1.9821426 , -0.27862012,  1.7827263 , -1.2839471 ],
       [-0.70371026, -1.4035337 ,  2.6128304 ,  0.75174505],
       [ 2.3730679 ,  0.3408125 , -0.25221348, -1.7284002 ]],
      dtype=float32), label_ids=array([0, 1, 0, 0, 1, 2, 0, 3, 2, 2, 0, 0, 2, 2, 2, 1, 0, 1, 1, 2, 0, 3,
       2, 1, 3, 0, 0, 2, 3, 2, 2, 2, 0, 0, 2, 2, 2, 3, 2, 3, 1, 2, 2, 3,
       0, 1, 2, 2, 0, 2, 2, 3, 3, 2, 1, 3, 0, 1, 3, 2, 2, 1, 2, 2, 2, 0,
       0, 2, 2, 1, 0, 0, 2, 2, 2, 1, 2, 3, 2, 1, 0, 1, 0, 2, 0, 2, 2, 1,
       2, 2, 2, 0, 2, 1, 0, 2, 0, 1, 2, 0, 3, 2, 2, 1, 3, 2, 1, 1, 2, 3,
       3, 3, 3, 2, 1, 2, 0, 3, 0, 3, 3, 3, 3, 2, 2, 0, 0, 2, 1, 2, 0, 0,
       2, 0, 0, 2, 2, 2, 3, 0, 2, 2, 2, 2, 3, 2, 2, 0, 2, 2, 2, 2, 2, 2,
       0, 2, 2, 2, 2, 1, 3, 0, 1, 0, 1, 0, 0, 1, 

In [8]:
actual_labels = tokenized_test['labels'].tolist()
predicted_classes = list(prediction.predictions.argmax(axis=1))
res = compute_metrics2(actual_labels, predicted_classes)

res

{'accuracy': 0.564935064935065,
 'precision': 0.5763598422650147,
 'recall': 0.564935064935065,
 'f1': 0.5678000457971303}